In [2]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\uday_nagisetti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
import pandas as pd

URL = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet"
df = pd.read_parquet(URL)

print(f"Number of records: {len(df)}")

Number of records: 3403766


In [18]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]

df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)

print(f"Filtered records: {len(df)}")

Filtered records: 3316216


In [19]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

model = LinearRegression()
model.fit(X_train, y_train)

print(f"Model intercept: {round(model.intercept_, 2)}")

Model intercept: 24.77


In [21]:
import yaml

# Specify the path to the metadata.yaml file in your root directory
file_path = "metadata.yaml"

# Open and read the YAML file from the local file system
with open(file_path, 'r') as file:
    content = file.read()

# Parse the YAML content
data = yaml.safe_load(content)

# Count lines in the YAML file
lines = content.splitlines()

# Print the number of lines
print(f"Number of lines in metadata.yaml: {len(lines)}")


Number of lines in metadata.yaml: 55


In [22]:
pip install mlflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\uday_nagisetti\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [28]:
import mlflow

# Create a new experiment if the previous one is deleted or doesn't exist
experiment_name = "new-nyc-taxi-experiment"
mlflow.set_experiment(experiment_name)

# Proceed with the rest of your code...


2025/04/02 12:11:19 INFO mlflow.tracking.fluent: Experiment with name 'new-nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/3', creation_time=1743576080771, experiment_id='3', last_update_time=1743576080771, lifecycle_stage='active', name='new-nyc-taxi-experiment', tags={}>

In [29]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
import pickle
import pandas as pd

# Load and prepare data
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]
df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)

# Features and labels
categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Set MLflow tracking
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("new-nyc-taxi-experiment")

with mlflow.start_run():
    # Log model
    mlflow.sklearn.log_model(model, "model", registered_model_name="lin_reg_model_hw3")

    # Log DictVectorizer as artifact
    with open("dv.pkl", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("dv.pkl")

    print("✅ Model and DictVectorizer logged and registered.")

2025/04/02 12:12:39 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'lin_reg_model_hw3' already exists. Creating a new version of this model...
2025/04/02 12:12:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lin_reg_model_hw3, version 2
Created version '2' of model 'lin_reg_model_hw3'.


✅ Model and DictVectorizer logged and registered.
🏃 View run sincere-fly-350 at: http://localhost:5000/#/experiments/3/runs/714993b39c2f47ee8635aced3f95861e
🧪 View experiment at: http://localhost:5000/#/experiments/3


In [31]:
import mlflow
import mlflow.sklearn
import os
import pickle
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from mlflow.tracking import MlflowClient
from mlflow.artifacts import download_artifacts

# Load and prepare data
df = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet")
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]
df[['PULocationID', 'DOLocationID']] = df[['PULocationID', 'DOLocationID']].astype(str)

# Features and labels
categorical = ['PULocationID', 'DOLocationID']
train_dicts = df[categorical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
y_train = df['duration'].values

# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Set MLflow tracking
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("new-nyc-taxi-experiment")

# Log model and DictVectorizer
with mlflow.start_run() as run:
    # Log model
    mlflow.sklearn.log_model(model, "model", registered_model_name="lin_reg_model_hw3")

    # Log DictVectorizer as artifact
    with open("dv.pkl", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("dv.pkl")
    
    print("✅ Model and DictVectorizer logged and registered.")

# Fetch the registered model
client = MlflowClient()
run_id = run.info.run_id
artifact_uri = client.get_run(run_id).info.artifact_uri
print(f"Artifact URI: {artifact_uri}")

# Download the model artifact locally
local_path = download_artifacts(run_id=run_id, artifact_path="model")

# Function to calculate size of downloaded model
def get_model_size(model_path):
    total_size = 0
    if os.path.exists(model_path):
        for dirpath, dirnames, filenames in os.walk(model_path):
            for filename in filenames:
                file_path = os.path.join(dirpath, filename)
                total_size += os.path.getsize(file_path)
    else:
        print(f"Model path does not exist: {model_path}")
    return total_size

# Get the size of the downloaded model artifact
model_size = get_model_size(local_path)

# Print model size
print(f"Downloaded model size: {model_size} bytes")



2025/04/02 12:14:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'lin_reg_model_hw3' already exists. Creating a new version of this model...
2025/04/02 12:14:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: lin_reg_model_hw3, version 3
Created version '3' of model 'lin_reg_model_hw3'.


✅ Model and DictVectorizer logged and registered.
🏃 View run clean-fowl-855 at: http://localhost:5000/#/experiments/3/runs/238dcbb7a70f4b2f817634186d9071c4
🧪 View experiment at: http://localhost:5000/#/experiments/3
Artifact URI: mlflow-artifacts:/3/238dcbb7a70f4b2f817634186d9071c4/artifacts


Downloaded model size: 5515 bytes
